In [1]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        gender = predict_gender(face)
        emotion = predict_emotion(face)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, f'Gender: {gender}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()





1/1 [==============================] - 0s 27ms/step


In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
age_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_age.prototxt", r"C:\Users\kaliv\Downloads\ikkada\age_net.caffemodel")
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")
# Function to predict age
def predict_age(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    age_model.setInput(blob)
    age = age_model.forward()[0][0]
    return age

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Function to predict dress color
def predict_dress_color(frame, x, y, w, h):
    roi = frame[y:y+h, x:x+w]
    blurred = cv2.GaussianBlur(roi, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

def predict_dress_color(frame, x, y, w, h):
    roi = frame[y:y+h, x:x+w]
    blurred = cv2.GaussianBlur(roi, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # Define color ranges for different colors
    color_ranges = {
        'red': ([0, 100, 100], [10, 255, 255]),
        'orange': ([11, 100, 100], [25, 255, 255]),
        'yellow': ([26, 100, 100], [40, 255, 255]),
        'green': ([41, 100, 100], [80, 255, 255]),
        'cyan': ([81, 100, 100], [100, 255, 255]),
        'blue': ([101, 100, 100], [130, 255, 255]),
        'purple': ([131, 100, 100], [155, 255, 255]),
        'pink': ([156, 100, 100], [175, 255, 255]),
        'brown': ([5, 50, 50], [20, 255, 255]),  # Adjust this range as needed
        'white': ([0, 0, 200], [180, 50, 255]),
        'black': ([0, 0, 0], [180, 255, 30]),
        'gray': ([0, 0, 31], [180, 30, 220])
    }

    for color_name, (lower, upper) in color_ranges.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)
        mask = cv2.inRange(hsv, lower, upper)
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
        if len(cnts) > 0:
            return color_name

    return None




# Capture video from webcam
cap = cv2.VideoCapture(0)
#address="https://192.168.1.42:8080"
#cap.open(address)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        age = predict_age(frame[y:y+h, x:x+w])
        gender = predict_gender(frame[y:y+h, x:x+w])
        emotion = predict_emotion(frame[y:y+h, x:x+w])
        dress_color = predict_dress_color(frame, x, y, w, h)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        #cv2.putText(frame, f'Age: {int(age)}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f'Gender: {gender}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        if dress_color:
            cv2.putText(frame, f'Dress Color: {dress_color}', (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 28ms/step


In [3]:
import cv2
import face_recognition
from deepface import DeepFace
import time

# Define age ranges
age_ranges = [
    (0, 5),
    (6, 10),
    (11, 15),
    (16, 20),
    (21, 25),
    (26, 30),
    (31, 35),
    (36, 40),
    (41, 45),
    (46, 50),
    (51, 55),
    (56, 60),
    (61, 65),
    (66, 70),
    (71, 75),
    (76, 80),
    (81, 85),
    (86, 90),
    (91, 95),
    (96, 100)
]

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Use face_recognition library to detect faces
    face_locations = face_recognition.face_locations(frame)
    
    # For each face, predict the age
    for top, right, bottom, left in face_locations:
        face_img = frame[top:bottom, left:right]
        
        # Analyze the face for age prediction
        try:
            age_prediction = DeepFace.analyze(face_img, enforce_detection=False)[0]['age']
            
            # Categorize age into predefined ranges
            age_category = "N/A"
            for age_range in age_ranges:
                if age_range[0] <= age_prediction <= age_range[1]:
                    age_category = f"{age_range[0]}-{age_range[1]}"
                    break
        except (ValueError, IndexError):
            age_category = "N/A"
        
        # Draw age prediction on the frame
        cv2.putText(frame, f'Age: {age_category}', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Draw rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Webcam Age Estimation', frame)
    
    # Introduce a delay for smoother camera feed
    time.sleep(0.05)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


Action: race: 100%|██████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.46it/s]


In [ ]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
age_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_age.prototxt", r"C:\Users\kaliv\Downloads\ikkada\age_net.caffemodel")
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Function to predict age
def predict_age(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    age_model.setInput(blob)
    age = age_model.forward()[0][0]
    return age

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Get user name and registration number
name = input("Enter your name: ")
reg_number = input("Enter your registration number: ")

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        age = predict_age(face)
        gender = predict_gender(face)
        emotion = predict_emotion(face)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, f'Name: {name}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f'Registration Number: {reg_number}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f'Gender: {gender}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import face_recognition
import os

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Function to collect known faces and their names
def collect_known_faces():
    known_face_encodings = []
    known_face_names = []

    # Example: Collect known faces from images
    known_faces_dir = r"C:\Users\kaliv\Downloads\ikkada\known_faces"
    for filename in os.listdir(known_faces_dir):
        image_path = os.path.join(known_faces_dir, filename)
        if os.path.isfile(image_path):
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_face_encodings.append(encoding)
            # Extract name from filename (assuming filename is in format "name.jpg")
            name = os.path.splitext(filename)[0]
            known_face_names.append(name)

    return known_face_encodings, known_face_names

# Load known faces and their names
known_face_encodings, known_face_names = collect_known_faces()

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Encode the face for recognition
        face_encodings = face_recognition.face_encodings(rgb_frame, [(y, x+w, y+h, x)])
        
        if len(face_encodings) > 0:
            # Face recognized
            matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])
            name = "Unknown"
            
            # Check if any known face matches
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
                
            # Display name instead of gender and emotion
            cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            # Face not recognized, predict gender and emotion
            gender = predict_gender(face)
            emotion = predict_emotion(face)

            cv2.putText(frame, f'Gender: {gender}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [9]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import face_recognition
import os

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Function to collect known faces and their names
def collect_known_faces():
    known_face_encodings = []
    known_face_names = []

    # Example: Collect known faces from images
    known_faces_dir = r"C:\Users\kaliv\Downloads\ikkada\known_faces"
    for filename in os.listdir(known_faces_dir):
        image_path = os.path.join(known_faces_dir, filename)
        if os.path.isfile(image_path):
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_face_encodings.append(encoding)
            # Extract name from filename (assuming filename is in format "name.jpg")
            name = os.path.splitext(filename)[0]
            known_face_names.append(name)

    return known_face_encodings, known_face_names

# Load known faces and their names
known_face_encodings, known_face_names = collect_known_faces()

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Encode the face for recognition
        face_encodings = face_recognition.face_encodings(rgb_frame, [(y, x+w, y+h, x)])
        
        if len(face_encodings) > 0:
            # Face recognized
            matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])
            name = "Unknown"
            
            # Check if any known face matches
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
                
            # Display name instead of gender and emotion
            cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            # Face not recognized, predict gender and emotion
            gender = predict_gender(face)
            emotion = predict_emotion(face)

            cv2.putText(frame, f'Gender: {gender}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [12]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import face_recognition
import os

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender



# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Function to collect known faces and their names
def collect_known_faces():
    known_face_encodings = []
    known_face_names = []

    # Example: Collect known faces from images
    known_faces_dir = r"C:\Users\kaliv\Downloads\ikkada\known_faces"
    for filename in os.listdir(known_faces_dir):
        image_path = os.path.join(known_faces_dir, filename)
        if os.path.isfile(image_path):
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_face_encodings.append(encoding)
            # Extract name from filename (assuming filename is in format "name.jpg")
            name = os.path.splitext(filename)[0]
            known_face_names.append(name)

    return known_face_encodings, known_face_names

# Load known faces and their names
known_face_encodings, known_face_names = collect_known_faces()

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Encode the face for recognition
        face_encodings = face_recognition.face_encodings(rgb_frame, [(y, x+w, y+h, x)])
        
        if len(face_encodings) > 0:
            # Face recognized
            matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])
            name = "Unknown"
            
            # Check if any known face matches
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
                
            # Display name instead of gender, age, and emotion
            cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            # Face not recognized, predict gender, age, and emotion
            gender = predict_gender(face)
            age = predict_age(face)
            emotion = predict_emotion(face)

            cv2.putText(frame, f'Name: Unknown', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Gender: {gender}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Age: {age}', (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [15]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import face_recognition
import os

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Always predict gender and emotion
        gender = predict_gender(face)
        emotion = predict_emotion(face)

        cv2.putText(frame, f'Name: Unknown', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f'Gender: {gender}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 254ms/step


In [23]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import face_recognition
import os

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Function to collect known faces and their names
def collect_known_faces():
    known_face_encodings = []
    known_face_names = []

    # Example: Collect known faces from images
    known_faces_dir = r"C:\Users\kaliv\Downloads\ikkada\known_faces"
    for filename in os.listdir(known_faces_dir):
        image_path = os.path.join(known_faces_dir, filename)
        if os.path.isfile(image_path):
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_face_encodings.append(encoding)
            # Extract name from filename (assuming filename is in format "name.jpg")
            name = os.path.splitext(filename)[0]
            known_face_names.append(name)

    return known_face_encodings, known_face_names

# Load known faces and their names
known_face_encodings, known_face_names = collect_known_faces()

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Encode the face for recognition
        face_encodings = face_recognition.face_encodings(frame, [(y, x+w, y+h, x)])
        
        if len(face_encodings) > 0:
            # Face recognized
            matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])
            name = "Unknown"
            
            # Check if any known face matches
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
                
            # Display name instead of gender and emotion
            cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            gender = predict_gender(face)
            emotion = predict_emotion(face)

            
            cv2.putText(frame, f'Gender: {gender}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    else:
            # Face not recognized, predict gender and emotion
            gender = predict_gender(face)
            emotion = predict_emotion(face)

            cv2.putText(frame, f'Name: Unknown', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Gender: {gender}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 27ms/step


In [28]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import face_recognition
import os

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Function to collect known faces and their names
def collect_known_faces():
    known_face_encodings = []
    known_face_names = []

    # Example: Collect known faces from images
    known_faces_dir = r"C:\Users\kaliv\Downloads\ikkada\known_faces"
    for filename in os.listdir(known_faces_dir):
        image_path = os.path.join(known_faces_dir, filename)
        if os.path.isfile(image_path):
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_face_encodings.append(encoding)
            # Extract name from filename (assuming filename is in format "name.jpg")
            name = os.path.splitext(filename)[0]
            known_face_names.append(name)

    return known_face_encodings, known_face_names

# Load known faces and their names
known_face_encodings, known_face_names = collect_known_faces()

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Encode the face for recognition
        face_encodings = face_recognition.face_encodings(frame, [(y, x+w, y+h, x)])
        
        if len(face_encodings) > 0:
            # Face recognized
            matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])
            
            if True in matches:
                # Known face recognized, get the name
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
            else:
                # Unknown face, set name as "Unknown"
                name = "Unknown"
                
            # Display name along with gender and emotion
            cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Gender: {gender}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            # Face not recognized, predict gender and emotion
            gender = predict_gender(face)
            emotion = predict_emotion(face)

            #cv2.putText(frame, f'Name: Unknown', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            #cv2.putText(frame, f'Gender: {gender}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            #cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [30]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np
import face_recognition
import os

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Function to collect known faces and their names
def collect_known_faces():
    known_face_encodings = []
    known_face_names = []

    # Example: Collect known faces from images
    known_faces_dir = r"C:\Users\kaliv\Downloads\ikkada\known_faces"
    for filename in os.listdir(known_faces_dir):
        image_path = os.path.join(known_faces_dir, filename)
        if os.path.isfile(image_path):
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)[0]
            known_face_encodings.append(encoding)
            # Extract name from filename (assuming filename is in format "name.jpg")
            name = os.path.splitext(filename)[0]
            known_face_names.append(name)

    return known_face_encodings, known_face_names

# Load known faces and their names
known_face_encodings, known_face_names = collect_known_faces()

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Encode the face for recognition
        face_encodings = face_recognition.face_encodings(frame, [(y, x+w, y+h, x)])
        
        if len(face_encodings) > 0:
            # Face recognized
            matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])
            
            if True in matches:
                # Known face recognized, get the name
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
                
                # Display name
                cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            else:
                # Unknown face, set name as "Unknown"
                name = "Unknown"
                
                # Predict gender and emotion
                gender = predict_gender(face)
                emotion = predict_emotion(face)

                # Display name, gender, and emotion
                cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(frame, f'Gender: {gender}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 28ms/step
